In [1]:
############################
#Author
#Lingjing Wang and Xiang Li
#Date: June 2020
############################

%matplotlib inline
import glob
import numpy as np
import os
import pandas as pd
from scipy.io import loadmat
import matplotlib.pylab as plt
import glob
from scipy.ndimage.interpolation import rotate
import scipy
import tensorflow as tf
import Utils
import h5py
from mpl_toolkits.mplot3d import Axes3D
colorsMap='jet'
import matplotlib
import matplotlib.cm as cmx

/home/donglin/anaconda3/envs/temp-seg37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/donglin/anaconda3/envs/temp-seg37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/donglin/anaconda3/envs/temp-seg37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/donglin/anaconda3/envs/te

/home/donglin/anaconda3/envs/temp-seg37/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/donglin/anaconda3/envs/temp-seg37/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/donglin/anaconda3/envs/temp-seg37/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/dong

In [2]:
#Please refer PointNet to download the part segmentation dataset
#download data at: https://github.com/charlesq34/pointnet/blob/master/part_seg/download_data.sh
ls_train=sorted(glob.glob("/home/donglin/Data/shapenet_part_seg_hdf5_data/hdf5_data/ply_data_train*.h5"))
ls_test=sorted(glob.glob("/home/donglin/Data/shapenet_part_seg_hdf5_data/hdf5_data/ply_data_test*.h5"))

In [3]:
print(ls_train)
print(ls_test)

['/home/donglin/Data/shapenet_part_seg_hdf5_data/hdf5_data/ply_data_train0.h5', '/home/donglin/Data/shapenet_part_seg_hdf5_data/hdf5_data/ply_data_train1.h5', '/home/donglin/Data/shapenet_part_seg_hdf5_data/hdf5_data/ply_data_train2.h5', '/home/donglin/Data/shapenet_part_seg_hdf5_data/hdf5_data/ply_data_train3.h5', '/home/donglin/Data/shapenet_part_seg_hdf5_data/hdf5_data/ply_data_train4.h5', '/home/donglin/Data/shapenet_part_seg_hdf5_data/hdf5_data/ply_data_train5.h5']
['/home/donglin/Data/shapenet_part_seg_hdf5_data/hdf5_data/ply_data_test0.h5', '/home/donglin/Data/shapenet_part_seg_hdf5_data/hdf5_data/ply_data_test1.h5']


In [4]:
def create_date(ls_train):
    D=[]
    L=[]
    P=[]
    for h5_filename in ls_train:
        print(h5_filename)
        f = h5py.File(h5_filename)
        data = f['data'][:]
        label = f['label'][:]
        pid=f['pid'][:]
        
        D.append(data)
        L.append(label)  
        P.append(pid)

    DD=np.concatenate(D, axis=0)
    LL=np.concatenate(L, axis=0)
    PP=np.concatenate(P, axis=0)
    
    # shuffle data
    DD,LL,PP=Utils.shuffle_data_pid(DD,LL,PP,seed=123)
    
    print(DD.shape,LL.shape,PP.shape)
    
    return DD, LL ,PP

In [5]:
# DD input point sets; LL categorical label; PP point label
DD, LL, PP = create_date(ls_train)

/home/donglin/Data/shapenet_part_seg_hdf5_data/hdf5_data/ply_data_train0.h5
/home/donglin/Data/shapenet_part_seg_hdf5_data/hdf5_data/ply_data_train1.h5
/home/donglin/Data/shapenet_part_seg_hdf5_data/hdf5_data/ply_data_train2.h5
/home/donglin/Data/shapenet_part_seg_hdf5_data/hdf5_data/ply_data_train3.h5
/home/donglin/Data/shapenet_part_seg_hdf5_data/hdf5_data/ply_data_train4.h5
/home/donglin/Data/shapenet_part_seg_hdf5_data/hdf5_data/ply_data_train5.h5
(12137, 2048, 3) (12137, 1) (12137, 2048)


In [15]:
cls=7 # chair is class 4
num_temp=10 # choose number of templates

# clas= Chair
ind=(LL==cls).reshape(-1,)
dd=DD[ind][:,:,:]
pp=PP[ind]
print(dd.shape, pp.shape)

(277, 2048, 3) (277, 2048)


In [10]:
for pc in pp:
    print(np.unique(pc))

[12 13 14 15]
[12 13 14]
[12 13 14]
[12 13 14]
[12 13 14]
[12 13 14]
[12 13 14 15]
[12 13 14]
[12 13 14 15]
[12 13 14 15]
[12 13 14 15]
[12 13 14 15]
[12 13 14 15]
[12 13 14 15]
[12 13 14]
[12 13 14 15]
[12 13 14]
[12 13 14]
[12 13 14 15]
[12 13 14]
[12 13 14]
[12 13 14]
[12 13 14]
[12 13 14 15]
[12 13 14]
[12 13 14 15]
[12 13 14]
[12 13 14]
[12 13 14 15]
[12 13 14]
[12 13 14]
[12 13 14]
[12 13 14]
[12 13 14]
[12 13 14 15]
[12 13 14]
[12 13 14]
[12 13 14 15]
[12 13 14 15]
[12 13 14]
[12 13 14 15]
[12 13 14]
[12 13 14]
[12 13 14]
[12 13 14 15]
[12 13 14 15]
[12 13 14]
[12 13 14]
[12 13 14]
[12 13 14]
[12 13 14 15]
[12 13 14 15]
[12 13 14]
[12 13 14 15]
[12 13 14]
[12 13 14]
[12 13 14]
[12 13 14]
[12 13 14 15]
[12 13 14]
[12 13 14 15]
[12 13 14]
[12 13 14 15]
[12 13 14]
[12 13 14]
[12 13 14]
[12 13 14]
[12 13 14]
[12 13 14]
[12 13 14]
[12 13 14 15]
[12 13 14]
[12 13 14]
[12 13 14]
[12 13 14]
[12 13 14]
[12 13 14]
[12 13 14]
[12 13 14 15]
[12 13 14 15]
[12 13 14]
[12 13 14]
[12 13 14]
[12

In [16]:
points_labels = np.load('/home/donglin/Github/acronym-partnet/data/combined_point_labels.npy')
shape_labels = np.load('/home/donglin/Github/acronym-partnet/data/combined_shape_labels.npy')
idx = (shape_labels==cls).reshape(-1,)
subset = points_labels[idx]
max_len = 0
for pc in points_labels:
    length = np.unique(pc).shape[0]
    print(np.unique(pc))
    if length > max_len:
        max_len = length
print(length)
print()

4

